In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd
import jieba
import re

In [2]:
class MyDataset(Dataset):
    def __init__(self, max_len, debug=True):
        super().__init__()
        df = pd.read_csv('../../datasets/THUCNews/train.csv')
        df = df.dropna()
        if debug:
            df = df.sample(2000).reset_index(drop=True)
        else:
            df = df.sample(50000).reset_index(drop=True)
        # 读取常用停用词
        stopwords = [line.strip() for line in open('../../stopwords/cn_stopwords.txt', 'r', encoding='utf-8').readlines()]
        sentences = []
        for title in df['title']:
            # 去除标点符号
            title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
            # jieba分词
            sentence_seged = jieba.cut(title.strip())
            outstr = ''
            for word in sentence_seged:
                if word != '\t' and word not in stopwords:
                    outstr += word
                    outstr += ' '
            if outstr != '':
                sentences.append(outstr)
        # 获取所有词(token), <pad>用来填充不满足max_len的句子
        token_list = ['<pad>'] + list(set(' '.join(sentences).split()))
        # token和index互转字典
        self.token2idx = {token: i for i, token in enumerate(token_list)}
        self.idx2token = {i: token for i, token in enumerate(token_list)}
        self.vocab_size = len(self.token2idx)

        self.inputs = []
        for sentence in sentences:
            tokens = sentence.split()
            input_ = [self.token2idx[token] for token in tokens]
            if len(input_) < max_len:
                self.inputs.append(input_ + [self.token2idx['<pad>']] * (max_len - len(input_)))
            else:
                self.inputs.append(input_[: max_len])

        self.labels = [[label] for label in df['label'].values.tolist()]
        self.n_class = len(df['label'].unique())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.LongTensor(self.inputs[idx]), torch.LongTensor(self.labels[idx])


class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_size, n_class):
        super().__init__()
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # 输入通道为1，卷成16通道输出，卷积核大小为(3*embed_size)，3类似于n-gram，可以换
        self.conv = nn.Conv2d(1, 16, (3, embed_size))
        self.dropout = nn.Dropout(0.2)
        # 输出头
        self.fc = nn.Linear(16, n_class)

    def forward(self, x):  # x: [batch_size * 句子长度]
        x = self.embedding(x)  # [batch_size * 句子长度 * embed_size]
        x = x.unsqueeze(1)  # [batch_size * 1 * 句子长度 * embed_size]，加一个维度，用于卷积层的输入
        x = self.conv(x)  # [batch_size * 16(卷积层输出通道数) * 8(卷积后的宽) * 1(卷积后的高)]
        x = x.squeeze(3)  # [batch_size * 16(卷积层输出通道数) * 8(卷积后的宽)] 压缩大小为1的维度
        x = torch.relu(x)  # 激活函数，尺寸不变
        x = torch.max_pool1d(x, x.size(2))  # 在每个通道做最大池化，[batch_size * 16(卷积层输出通道数) * 1]
        x = x.squeeze(2)  # 压缩维度2，[batch_size * 16(卷积层输出通道数)]
        x = self.dropout(x)  # dropout，尺寸不变
        logits = self.fc(x)  # 全连接输出头，[batch_size * n_class]
        return logits

In [3]:
dataset = MyDataset(max_len=10)  # 构造长度为10的句子输入，超过10的句子切掉，不足10的补<pad>
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
model = TextCNN(vocab_size=dataset.vocab_size, embed_size=128, n_class=dataset.n_class)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
for epoch in range(100):
    for feature, target in dataloader:
        optimizer.zero_grad()
        logits = model(feature)
        loss = criterion(logits, target.squeeze())
        loss.backward()
        optimizer.step()
    print('epoch:', epoch + 1, ', loss:', loss.item())

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/d1/4_gsqv2176z583_7rmpm27lh0000gn/T/jieba.cache
Loading model cost 0.483 seconds.
Prefix dict has been built successfully.


epoch: 1 , loss: 2.0988645553588867
epoch: 2 , loss: 1.6732847690582275
epoch: 3 , loss: 1.7872973680496216
epoch: 4 , loss: 1.632114291191101
epoch: 5 , loss: 2.034989833831787
epoch: 6 , loss: 1.2769547700881958
epoch: 7 , loss: 0.8881691694259644
epoch: 8 , loss: 0.7390443682670593
epoch: 9 , loss: 0.9840537905693054
epoch: 10 , loss: 0.6907981634140015
epoch: 11 , loss: 0.8755264282226562
epoch: 12 , loss: 0.47071027755737305
epoch: 13 , loss: 0.2673209309577942
epoch: 14 , loss: 0.4598577320575714
epoch: 15 , loss: 0.3407303988933563
epoch: 16 , loss: 0.4497127830982208
epoch: 17 , loss: 0.10531115531921387
epoch: 18 , loss: 0.37725019454956055
epoch: 19 , loss: 0.4173489809036255
epoch: 20 , loss: 0.2735407054424286
epoch: 21 , loss: 0.11588075011968613
epoch: 22 , loss: 0.13563832640647888
epoch: 23 , loss: 0.07552756369113922
epoch: 24 , loss: 0.10603988170623779
epoch: 25 , loss: 0.23832593858242035
epoch: 26 , loss: 0.04341195523738861
epoch: 27 , loss: 0.20184533298015594
ep

In [4]:
df = pd.read_csv('../../datasets/THUCNews/train.csv')
predict = model(feature).max(1)[1].tolist()
for i in range(len(feature.tolist())):
    print(' '.join([dataset.idx2token[idx] for idx in feature.tolist()[i]]),
          '---> label:',
          dict(zip(df['label'], df['class']))[predict[i]])

盘整 改向 趋势 <pad> <pad> <pad> <pad> <pad> <pad> <pad> ---> label: 股票
地量 频出 面对 <pad> <pad> <pad> <pad> <pad> <pad> <pad> ---> label: 股票
低 库存 国际 粮价 持续 上涨 动力 <pad> <pad> <pad> ---> label: 财经
快乐 男声 曝 黑幕 疑云 传 刘心 第五名 已定 图 ---> label: 娱乐
李孝利 月 中 发行 第张 专辑 三度 涉嫌 抄袭 图 ---> label: 家居
赫斯特 国际 业务 发起 亿美元 要约 <pad> <pad> <pad> <pad> ---> label: 股票
谷歌风 投向 付费 搜索 服务 投资 万美元 <pad> <pad> <pad> ---> label: 科技
上海 证监局 部署 今年 期货 监管 十大 工作 <pad> <pad> ---> label: 财经
种 装修 风格 助 打造 个性 客厅 组图 <pad> <pad> ---> label: 家居
打工仔 冒充 美女 网上 诈骗 初中 女生 万 <pad> <pad> ---> label: 社会
廖斌 谈 做好 网站 需要 强悍 身体 <pad> <pad> <pad> ---> label: 科技
零碳 家庭 倡议书 <pad> <pad> <pad> <pad> <pad> <pad> <pad> ---> label: 家居
限量 台徕 卡 发布 版 <pad> <pad> <pad> <pad> <pad> ---> label: 科技
性能 出众 单反 辅 机理 光送 卡 <pad> <pad> <pad> ---> label: 科技
大学 设置 拆迁 专业 解 不了 不公 <pad> <pad> <pad> ---> label: 教育
大厦 凌晨 起火 保安 爬层 逐户 叫醒 居民 <pad> <pad> ---> label: 社会
